In [221]:
import requests
import json
import os
import pandas as pd
import time 
import plotly.express as px

In [222]:
API_Key = "-----------"
URL = "https://api.rawg.io/api/games"

In [223]:
def get_game_info(game):
    game_id = result["id"]
    game_name = result["name"]
    game_release_date = result["released"]
    game_rating = result["rating"]
    game_metacritic = result["metacritic"]
    game_playtime = result["playtime"]
    
    return game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime

        

In [224]:
df = pd.DataFrame(columns=["game_id", "game_name", "game_release_date", "game_rating", \
                           "game_metacritic","game_playtime"])

In [225]:
j = 0
for i in range(1,100):
    querystring = {"key": API_Key, "page": i}
    try:
        response = requests.request("GET", URL, params=querystring)
        response = response.json()
    except JSONDecodeError:
        break
    for result in response["results"]:
        game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime = get_game_info(result)
        df = df.append({"game_id": game_id, "game_name": game_name, "game_release_date": game_release_date, \
                        "game_rating": game_rating, "game_metacritic": game_metacritic, \
                        "game_playtime": game_playtime}, ignore_index=True)
  

In [226]:
df

,game_id,game_name,game_release_date,game_rating,game_metacritic,game_playtime
0,3498,Grand Theft Auto V,2013-09-17,4.47,92,73
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.66,92,46
2,4200,Portal 2,2011-04-18,4.62,95,11
3,5286,Tomb Raider (2013),2013-03-05,4.05,86,10
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81,65
...,...,...,...,...,...,...
1975,51081,Beholder 2,2018-12-04,3.71,76,5
1976,28414,FIFA 12,2011-09-27,3.39,85,0
1977,20960,The Plan,2014-02-14,3.25,50,1
1978,20218,Canyon Capers,2014-04-25,1.87,None,3


In [239]:
px.histogram(df, x="game_metacritic", nbins=25, title="Game Rating Distribution")


In [228]:

x = px.scatter(df, x="game_rating", y="game_metacritic", title = "Fans vs. Metacritic", trendline="ols", 
           trendline_color_override="green")

In [236]:
x

In [234]:
#x.write_image("metacritic_vs_fans.png")

In [235]:
#x.to_image(format="png")

In [ ]:
#save px scatter plot as png


# Exporting to a database

In [174]:
import psycopg2 as ps
import pandas as pd

In [175]:
df.head()

,game_id,game_name,game_release_date,game_rating,game_metacritic,game_playtime
0,3498,Grand Theft Auto V,2013-09-17,4.47,92,73
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.66,92,46
2,4200,Portal 2,2011-04-18,4.62,95,11
3,5286,Tomb Raider (2013),2013-03-05,4.05,86,10
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81,65


In [176]:
#change values in game_id from int to string
df["game_id"] = df["game_id"].astype(str)


In [177]:
df["game_id"][1]

'3328'

In [178]:
#Check for null values in df
df.isnull().sum()

game_id                0
game_name              0
game_release_date      5
game_rating            0
game_metacritic      402
game_playtime          0
dtype: int64

In [179]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [195]:
def delete_table(curr, table_name):
    delete_query = "DROP TABLE IF EXISTS {}".format(table_name)
    curr.execute(delete_query)

In [181]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS games(
                                game_id VARCHAR(225),
                                game_name VARCHAR(225) NOT NULL,
                                game_release_date DATE Default NULL,
                                game_rating FLOAT NOT NULL,
                                game_metacritic INTEGER,
                                game_playtime INTEGER NOT NULL,
                                PRIMARY KEY ("game_id")
                                );""")

    curr.execute(create_table_command)

In [182]:
def check_if_game_exists(curr, game_id): 
    query = ("""SELECT game_id FROM games WHERE game_id = %s""")

    curr.execute(query, (game_id,))
    return curr.fetchone() is not None


In [183]:
def update_row(curr, game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime):
    query = ("""UPDATE games 
                SET game_name = %s, 
                    game_release_date = %s, 
                    game_rating = %s, 
                    game_metacritic = %s, 
                    game_playtime = %s 
                WHERE game_id = %s""")
    vars_to_insert = (game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime)
    curr.execute(query, vars_to_insert)
    

In [184]:
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=["game_id", "game_name", "game_release_date", "game_rating", 
                                   "game_metacritic","game_playtime"])
    for i, row in df.iterrows():
        if check_if_game_exists(curr, row['game_id']): # If video already exists then we will update
            update_row(curr,row['game_id'],row['game_name'],row['game_release_date'],row['game_rating'],
                           row['game_metacritic'],row['game_playtime'])
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [185]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['game_id'],row['game_name'],row['game_release_date'],row['game_rating'],
                               row['game_metacritic'],row['game_playtime'] )


In [186]:
def insert_into_table(curr, game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime):
    query = ("""INSERT INTO games (game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime) 
                VALUES (%s, %s, %s, %s, %s, %s)""")
    
    vars_to_insert = (game_id, game_name, game_release_date, game_rating, game_metacritic, game_playtime)
    curr.execute(query, vars_to_insert)
    

In [187]:
#get column names and store in variable column_names
column_names = df.columns.tolist()

In [188]:
column_names

['game_id',
 'game_name',
 'game_release_date',
 'game_rating',
 'game_metacritic',
 'game_playtime']

In [189]:
## Main
#database credentials

host_name = '--------------------'
dbname = '-----'
port = '5432'
username = '-----' 
password = '-----------'
conn = None

conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [196]:
delete_table(curr, "games")
conn.commit()

In [197]:
create_table(curr)
conn.commit()

In [192]:
#update data for existing videos

new_games_df = update_db(curr,df)
conn.commit()

In [198]:
new_games_df.head()

,game_id,game_name,game_release_date,game_rating,game_metacritic,game_playtime
0,3498,Grand Theft Auto V,2013-09-17,4.47,92,73
1,3328,The Witcher 3: Wild Hunt,2015-05-18,4.66,92,46
2,4200,Portal 2,2011-04-18,4.62,95,11
3,5286,Tomb Raider (2013),2013-03-05,4.05,86,10
4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81,65


In [199]:
#insert new videos into db table

append_from_df_to_db(curr, new_games_df)
conn.commit()

In [203]:
#view data in db table
"""curr.execute("SELECT * FROM games")
print(curr.fetchall())  """

'curr.execute("SELECT * FROM games")\nprint(curr.fetchall())  '